In [35]:
import pandas as pd
train_data = pd.read_csv('train.csv')
print("Train data:\n", train_data.dtypes)
holiday_data = pd.read_csv('holidays_events.csv')
print("Holiday data:\n", holiday_data.dtypes)
oil_data = pd.read_csv('oil.csv')
print("Oil data:\n", oil_data.dtypes)
store_data = pd.read_csv('stores.csv')
print("Store data:\n", store_data.dtypes)

Train data:
 id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object
Holiday data:
 date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object
Oil data:
 date           object
dcoilwtico    float64
dtype: object
Store data:
 store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object


In [36]:
import mysql.connector
from mysql.connector import errorcode

# connect to my sql localhost
conn = mysql.connector.connect(user='root', password='sql',
                              host='localhost')

# creating pseudo tables to later transfer data from csv in
TABLES = {}

TABLES['train'] = (
    "CREATE TABLE `train` ("
    "   `id` int(16),"
    "   `date` date,"
    "   `store_nbr` int(16),"
    "   `family` varchar(255),"
    "   `sales` float(16),"
    "   `onpromotion` int(16),"
    "  PRIMARY KEY (`id`)"
    ")"
)

TABLES['holidays_events'] = (
    "CREATE TABLE `holidays` ("
    "   `date` date,"
    "   `type` varchar(255),"
    "   `locale` varchar(255),"
    "   `locale_name` varchar(255),"
    "   `description` varchar(255),"
    "   `transferred` int(16)"
    ")"
)

TABLES['oil'] = (
    "CREATE TABLE `oil` ("
    "   `date` date,"
    "   `dcoilwtico` float(16)"
    ")"
)

TABLES['stores'] = (
    "CREATE TABLE `stores` ("
    "   `store_nbr` int(16),"
    "   `city` varchar(255),"
    "   `state` varchar(255),"
    "   `type` varchar(255),"
    "   `cluster` int(16)"
    ")"
)

TABLES['transactions'] = (
    "CREATE TABLE `transactions` ("
    "   `date` date,"
    "   `store_nbr` int(16),"
    "   `transactions` int(16)"
    ")"
)

cur = conn.cursor()

DB_NAME = 'sales'
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
try:
    cur.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cur)
        print("Database {} created successfully.".format(DB_NAME))
        conn.database = DB_NAME
    else:
        print(err)
        exit(1)

Database sales does not exists.
Database sales created successfully.


In [37]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cur.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print("Error: ", err.msg)
    else:
        print("OK")

Creating table train: OK
Creating table holidays_events: OK
Creating table oil: OK
Creating table stores: OK


In [52]:
import pandas as pd
train_data = pd.read_csv('train.csv')
for i,row in train_data.iterrows():
    #here %S means string values 
    sql = "INSERT INTO sales.train VALUES (%s,%s,%s,%s,%s,%s)"
    cur.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    conn.commit()

holiday_data = pd.read_csv('holidays_events.csv')
for i,row in holiday_data.iterrows():
    #here %S means string values 
    sql = "INSERT INTO sales.holidays VALUES (%s,%s,%s,%s,%s,%s)"
    cur.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    conn.commit()

oil_data = pd.read_csv('oil.csv')
oil_data = oil_data.astype(object).where(pd.notnull(oil_data), None)
for i,row in oil_data.iterrows():
    #here %S means string values 
    sql = "INSERT INTO sales.oil VALUES (%s,%s)"
    cur.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    conn.commit()

store_data = pd.read_csv('stores.csv')
for i,row in store_data.iterrows():
    #here %S means string values 
    sql = "INSERT INTO sales.stores VALUES (%s,%s,%s,%s,%s)"
    cur.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    conn.commit()

transaction_data = pd.read_csv('transactions.csv')
for i,row in transaction_data.iterrows():
    #here %S means string values 
    sql = "INSERT INTO sales.transactions VALUES (%s,%s,%s)"
    cur.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    conn.commit()

In [62]:
conn.close()